In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from tqdm import tqdm
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import gurobipy as gp
from gurobipy import GRB
pd.options.display.float_format = "{:,.2f}".format
from itertools import product
pd.set_option('display.max_columns', None)

In [2]:
qdf = pd.read_csv('data.csv', sep=';' , encoding='latin-1')
qdf['datetime'] = pd.to_datetime(qdf['ISO TIMESTAMP'])
qdf = qdf.rename(columns={'HQ - Bes.québécois(BQ) - Mesure': 'Load'})
qdf['Load'] /= 200000
qdf = qdf.drop(qdf.columns[[0,1,3,4]], axis=1)
qdf = qdf.reindex(columns=['datetime', 'Load'])
qdf.set_index('datetime', inplace=True)
df = qdf.resample('H').sum()
df = df[df.index.month.isin([1, 2, 3, 12])]
df['Load'] = df['Load'].replace(0, pd.NaT).fillna(method='ffill')
df['DateTime'] = df.index
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Hour'] = df.index.hour
p = pd.read_csv('HQP.csv', sep=',')
p['datetime'] = pd.to_datetime(p['Date-Hour'], format='%m/%d/%Y %H:%M')
p = p.drop(columns='Date-Hour')
p = p.set_index('datetime')
tmp = pd.date_range(start="1/1/2013", end="3/31/2019", freq='H').difference(p.index)
for i in tmp:
    p.loc[i,'Price'] = p.loc[i - pd.Timedelta(hours=1),'Price']
mask = (p.index.month.isin([4,5,6,7,8,9,10,11]))
p = p[~mask]
# reference: https://www.nyiso.com/custom-reports
r = pd.read_csv('R.csv', sep=',')
# reference: https://re.jrc.ec.europa.eu/pvg_tools/en/tools.html#PVP, PV technology:CIS, Installed peak PV power [kWp]: 2, System loss [%]: 14, Solar radiation databases: PVGIS-ERA5, Location: 45.551, -73.602, Mounting type: Two-axis
r['datetime'] = pd.to_datetime(r['time'], format='%Y%m%d:%H%M')
r = r.drop(r.columns[[0,2,3,5,6]], axis=1)
r['P'] = r['P'].shift(-4)
r['P'] /= 1000
mask = (r['datetime'].dt.year == 2019) & (r['datetime'].dt.month.isin([4,5,6,7,8,9,10,11,12]))
r = r[~mask]
mask = (r['datetime'].dt.month.isin([4,5,6,7,8,9,10,11]))
r = r[~mask]
r.reset_index(inplace = True, drop=True)
df.reset_index(inplace = True, drop=True)
df.loc[:,'R'] = r.loc[:,'P']
df.loc[:,'T'] = r.loc[:,'T2m']
df.loc[:,'P'] = p.loc[:,'Price'].to_numpy()/1000
columns = 'DateTime,Year,Month,Day,Hour,Load,PV,EV,BAT,DR,R,P,T,WCO0,WCO1,WCO2,WCO3,FXD0,FXD1,FXD2,FXD3'.split(',')
data = pd.DataFrame(columns=columns)
for PV in [False, True]:
    for EV in [False, True]:
        for BAT in [False, True]:
            for DR in [False, True]:
                qdf = pd.DataFrame(index=df.index, columns=columns)
                qdf.loc[:, ['DateTime', 'Load', 'Year', 'Month', 'Day', 'Hour', 'R', 'P', 'T']] = df.loc[:, ['DateTime', 'Load', 'Year', 'Month', 'Day', 'Hour', 'R', 'P', 'T']]
                qdf.PV = PV
                qdf.EV = EV
                qdf.BAT = BAT
                qdf.DR = DR
                data = pd.concat([data, qdf])
data.reset_index(drop=True, inplace = True)
# id = data.query(f"PV == False and EV == False and BAT == False and DR == False").index
# tmp = data.loc[id,'Load']
# data.loc[id,['WCO0', 'WCO1', 'WCO2', 'WCO3', 'FXD0', 'FXD1', 'FXD2', 'FXD3']] = tmp
data[['Load', 'R', 'P', 'T']] = data[['Load', 'R', 'P', 'T']].astype(float)
d = {}
d["U_c"] = 10
d["U_d"] = 10
d["eta_c"] = 0.95
d["eta_d"] = 0.95
d["alpha"] = 1  #0.21<alpha<1.73
d["T"] = 24
d["LL"] = 40
d["PC"] = 1.35*5+9
d["PR"] = {}
d["PH"] = {}
d["PR"]['WCO'] = 0.06319
d["PR"]['FXD'] = 0.04449
d["PH"]['WCO'] = 0.09749
d["PH"]['FXD'] = 0.07650
d["PP"] = 0.51967
d["P"] = {}
d["P"]['WCO'] = d["PR"]['WCO'] * np.ones(d['T'])
d["P"]['FXD'] = d["PR"]['FXD'] * np.ones((4,d['T']))
d["P"]['FXD'][1][[6,7,8]] = d["PP"]
d["P"]['FXD'][2][[16,17,18,19]] = d["PP"]
d["P"]['FXD'][3][[6,7,8,16,17,18,19]] = d["PP"]
db = {}
db['d'] = d
db['data'] = data
with open('db.pkl', 'wb') as handle:
    pickle.dump(db, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
data.head(5)

,DateTime,Year,Month,Day,Hour,Load,PV,EV,BAT,DR,R,P,T,WCO0,WCO1,WCO2,WCO3,FXD0,FXD1,FXD2,FXD3
0,2013-01-01 00:00:00,2013,1,1,0,1.47,False,False,False,False,0.00,0.00,-2.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-01 01:00:00,2013,1,1,1,1.45,False,False,False,False,0.00,0.01,-1.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-01 02:00:00,2013,1,1,2,1.45,False,False,False,False,0.00,-0.01,-0.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01 03:00:00,2013,1,1,3,1.46,False,False,False,False,0.00,-0.00,-0.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-01 04:00:00,2013,1,1,4,1.49,False,False,False,False,0.00,0.01,-0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def model(L, R, E_max, E_min, A, V, B_max, B_min, D_max, d, sq, type, peaks, report):
    episodes = {'peak': peaks, 'off-peak': [i for i in range(24) if i not in peaks]}
    m = gp.Model('home_management')
    m.Params.LogToConsole = 0
    # Decision Vars
    x_GL   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^GL')
    x_GB   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^GB')
    x_GE   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^GE')
    x_RL   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^RL')
    x_RB   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^RB')
    x_RE   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^RE')
    x_RG   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^RG')
    x_BL   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^BL')
    x_EL   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^EL')
    x_BG   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^BG')
    x_EG   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^EG')
    x_D    = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^D')
    b_E    = m.addVars(d['T']+1, vtype=GRB.CONTINUOUS, name='b^E')
    b_B    = m.addVars(d['T']+1, vtype=GRB.CONTINUOUS, name='b^B')
    x_EG   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^EG')
    x_G    = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^G', lb=-GRB.INFINITY)
    x_SQ   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^SQ')
    x_SQ1   = m.addVars(d['T'], vtype=GRB.CONTINUOUS, name='x^SQ1', lb=-GRB.INFINITY)
    x_Lpos   = m.addVar(vtype=GRB.CONTINUOUS, name='x^Lpos')
    x_Lpos1   = m.addVar(vtype=GRB.CONTINUOUS, name='x^Lpos1', lb=-GRB.INFINITY)
    # Constraints
    m.addConstrs((x_GL[t] + x_GB[t] +  x_GE[t] - (x_RG[t] + d['eta_d'] * (x_BG[t] + x_EG[t])) == x_G[t] for t in range(d['T'])), name='x_G')
    m.addConstrs((x_GL[t] + d['eta_d'] * x_BL[t] + d['eta_d'] * x_EL[t] + x_RL[t] + x_D[t] == L[t] for t in range(d['T'])), name='c_load')
    m.addConstrs((x_D[t] <= D_max * L[t] for t in range(d['T'])), name='c_D_max')
    m.addConstrs((x_SQ1[t] == sq[t] - x_G[t] for t in range(d['T'])), name='c_x_SQ1')
    m.addConstrs((x_SQ[t] == gp.max_([x_SQ1[t],0.0])for t in range(d['T'])), name='c_x_SQ')
    m.addConstr((x_Lpos1 == gp.quicksum(x_G[t] for t in range(d['T']))-d['LL']), name='c_x_Lpos1')
    m.addConstr((x_Lpos == gp.max_([x_Lpos1,0.0])), name='c_Lpos')
    m.addConstr((b_E[0] == b_E[24] ), name='c_init_EV')
    m.addConstr((b_B[0] == b_B[24] ), name='c_init_Bat')
    m.addConstrs((x_BL[t] + x_BG[t] <= b_B[t] for t in range(d['T'])), name='c_bat_state')
    m.addConstrs((x_EL[t] + x_EG[t] <= b_E[t] * A[t] for t in range(d['T'])), name='c_ev_state')
    m.addConstrs((x_RE[t] + x_GE[t] <= E_max * A[t] for t in range(d['T'])), name='c_ev_state_2')
    m.addConstrs((x_BL[t] + x_BG[t] <= d['U_d'] for t in range(d['T'])), name='c_b_dch_max')
    m.addConstrs((x_EL[t] + x_EG[t] <= d['U_d'] for t in range(d['T'])), name='c_e_dch_max')
    m.addConstrs((x_GB[t] + x_RB[t] <= d['U_c'] for t in range(d['T'])), name='c_b_ch_max')
    m.addConstrs((x_GE[t] + x_RE[t] <= d['U_c'] for t in range(d['T'])), name='c_e_ch_max')
    m.addConstrs((b_B[t] <= B_max for t in range(d['T']+1)), name='c_b_B_max')
    m.addConstrs((b_B[t] >= B_min for t in range(d['T']+1)), name='c_b_B_min')
    m.addConstrs((b_E[t] <= E_max for t in range(d['T']+1)), name='c_b_E_max')
    m.addConstrs((b_E[t] >= E_min for t in range(d['T']+1)), name='c_b_E_min')
    m.addConstrs((x_RL[t] + x_RB[t] + x_RE[t] + x_RG[t] <= R[t] for t in range(d['T'])), name='c_gen_cap')
    m.addConstrs((b_B[t] + d['eta_c'] * (x_GB[t] + x_RB[t]) - (x_BL[t] + x_BG[t]) == b_B[t+1] for t in range(d['T'])), name='c_next_state_b_B')
    m.addConstrs((b_E[t] + d['eta_c'] * (x_GE[t] + x_RE[t]) - (x_EL[t] + x_EG[t]) - (V[t] / d['eta_d']) == b_E[t+1] for t in range(d['T'])), name='c_next_state_b_E')
    # Objective
    if type == 'WCO':
        m.setObjective(gp.quicksum(d["PR"]['WCO'] * x_G[t] + d['alpha'] * x_D[t] * x_D[t] for t in range(d['T'])) - gp.quicksum(d['PP'] * x_SQ[t] for t in episodes['peak']) + (d["PH"]['WCO']-d["PR"]['WCO'])*x_Lpos , GRB.MINIMIZE)
    else:
        m.setObjective(gp.quicksum(d["PR"]['FXD'] * x_G[t] for t in episodes['off-peak']) + gp.quicksum(d['PP'] * x_G[t] for t in episodes['peak']) + gp.quicksum(d['alpha'] * x_D[t] * x_D[t] for t in range(d['T'])) + (d["PH"]['FXD']-d["PR"]['FXD'])*(gp.quicksum(x_G[t] for t in episodes['off-peak'])-d['LL']), GRB.MINIMIZE)

    # Optimize
    m.optimize()
    sol = [x_GL[i].x+x_GB[i].x+x_GE[i].x-x_RG[i].x-x_BG[i].x-x_EG[i].x for i in range(d['T'])]
    if report:
        clmn = 'x_GL,x_GB,x_GE,x_RL,x_RB,x_RE,x_RG,x_BL,x_EL,x_BG,x_EG,x_D,x_G,b_E,b_B,L,SQ,Peak Reduction,CPP,COST'.split(',')
        df = pd.DataFrame(data = np.zeros((25, len(clmn))), columns = clmn)
        df.loc[0:23,'x_GL'] = [x_GL[i].x for i in range(24)]
        df.loc[0:23,'x_GB'] = [x_GB[i].x for i in range(24)]
        df.loc[0:23,'x_GE'] = [x_GE[i].x for i in range(24)]
        df.loc[0:23,'x_RL'] = [x_RL[i].x for i in range(24)]
        df.loc[0:23,'x_RB'] = [x_RB[i].x for i in range(24)]
        df.loc[0:23,'x_RE'] = [x_RE[i].x for i in range(24)]
        df.loc[0:23,'x_RG'] = [x_RG[i].x for i in range(24)]
        df.loc[0:23,'x_BL'] = [x_BL[i].x for i in range(24)]
        df.loc[0:23,'x_EL'] = [x_EL[i].x for i in range(24)]
        df.loc[0:23,'x_BG'] = [x_BG[i].x for i in range(24)]
        df.loc[0:23,'x_EG'] = [x_EG[i].x for i in range(24)]
        df.loc[0:23,'x_D']  = [x_D[i].x for i in range(24)]
        df.loc[0:23,'x_G']  = [x_G[i].x for i in range(24)]
        df.loc[0:24,'b_B']  = [b_B[i].x for i in range(25)]
        df.loc[0:24,'b_E']  = [b_E[i].x for i in range(25)]
        df.loc[:23, 'L']    = L
        df.loc[:23, 'CPP']  = [sq[i] - x_G[i].x for i in range(d['T'])]
        df.loc[:23, 'SQ']   = sq
        df.loc[:23, 'Peak Reduction'] = [x_SQ[i].x for i in range(2)]
        return df, sol
    else:
        return sol
   

In [5]:
yy = data['Year'].unique()
for year in yy:
    mm = data.query(f'Year == {year}')['Month'].unique()
    for month in mm:
        dd = data.query(f'Year == {year} and Month == {month}')['Day'].unique()
        for day in dd:
            for PV in [False, True]:
                for EV in [False, True]:
                    for BAT in [False, True]:
                        for DR in [False, True]:
                            print(f"**Year == {year} and Month == {month} and Day == {day} and PV == {PV} and EV == {EV} and BAT == {BAT} and DR == {DR}**")
                            id = data.query(f"Year == {year} and Month == {month} and Day == {day} and PV == {PV} and EV == {EV} and BAT == {BAT} and DR == {DR}").index
                            L = data.loc[id,'Load'].to_numpy()
                            if PV:
                                R = data.loc[id,'R'].to_numpy()
                            else:
                                R = np.zeros(d['T'])

                            if EV:
                                E_max, E_min, A, V = 60, 15, np.ones(d['T']), np.zeros(d['T'])
                                A[9:17], V[9:17] =0, 10/8
                                # add EV cost compensation to the costs of the consumer
                            else:
                                E_max, E_min, A, V = 0, 0, np.zeros(d['T']), np.zeros(d['T'])

                            B_min, B_max = 0, 0
                            if BAT:
                                B_max = 60
                                
                            if DR:
                                D_max = 0.25
                            else:
                                D_max = 0 
                            #to calc sq solve the model with act 0 and any sq for this, because consumer does not care about the diff btw load and sq
                            sq = model(L, R, E_max, E_min, A, V, B_max, B_min, D_max, d, np.zeros(d['T']), 'WCO', [], False)
                            for act in [0,1,2,3]:
                                if act == 0:
                                    peaks = []
                                elif act == 1:
                                    peaks = [6,7,8]
                                elif act == 2:
                                    peaks = [16,17,18,19]
                                else:
                                    peaks = [6,7,8,16,17,18,19]
                                data.loc[id,f"WCO{act}"] = model(L, R, E_max, E_min, A, V, B_max, B_min, D_max, d, sq, 'WCO', peaks, False)
                                data.loc[id,f"FXD{act}"] = model(L, R, E_max, E_min, A, V, B_max, B_min, D_max, d, sq, 'FXD', peaks, False)

**Year == 2013 and Month == 1 and Day == 1 and PV == False and EV == False and BAT == False and DR == False**
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-09
**Year == 2013 and Month == 1 and Day == 1 and PV == False and EV == False and BAT == False and DR == True**
**Year == 2013 and Month == 1 and Day == 1 and PV == False and EV == False and BAT == True and DR == False**
**Year == 2013 and Month == 1 and Day == 1 and PV == False and EV == False and BAT == True and DR == True**
**Year == 2013 and Month == 1 and Day == 1 and PV == False and EV == True and BAT == False and DR == False**
**Year == 2013 and Month == 1 and Day == 1 and PV == False and EV == True and BAT == False and DR == True**
**Year == 2013 and Month == 1 and Day == 1 and PV == False and EV == True and BAT == True and DR == False**
**Year == 2013 and Month == 1 and Day == 1 and PV == False and EV == True and BAT == True and DR == True**
**Year == 2013 and Month == 1 and Day == 

In [6]:
data.head(5)

,DateTime,Year,Month,Day,Hour,Load,PV,EV,BAT,DR,R,P,T,WCO0,WCO1,WCO2,WCO3,FXD0,FXD1,FXD2,FXD3
0,2013-01-01 00:00:00,2013,1,1,0,1.47,False,False,False,False,0.00,0.00,-2.08,1.47,1.47,1.47,1.47,1.47,1.47,1.47,1.47
1,2013-01-01 01:00:00,2013,1,1,1,1.45,False,False,False,False,0.00,0.01,-1.22,1.45,1.45,1.45,1.45,1.45,1.45,1.45,1.45
2,2013-01-01 02:00:00,2013,1,1,2,1.45,False,False,False,False,0.00,-0.01,-0.82,1.45,1.45,1.45,1.45,1.45,1.45,1.45,1.45
3,2013-01-01 03:00:00,2013,1,1,3,1.46,False,False,False,False,0.00,-0.00,-0.55,1.46,1.46,1.46,1.46,1.46,1.46,1.46,1.46
4,2013-01-01 04:00:00,2013,1,1,4,1.49,False,False,False,False,0.00,0.01,-0.60,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.49


In [7]:
data['Saving1'] = data['WCO0'] - data['WCO1']
data['Saving1'] = data['Saving1'].clip(lower=0)
data.loc[~data['Hour'].isin([6,7,8]),'Saving1'] = 0
data['Saving2'] = data['WCO0'] - data['WCO2']
data['Saving2'] = data['Saving2'].clip(lower=0)
data.loc[~data['Hour'].isin([16,17,18,19]),'Saving2'] = 0
data['Saving3'] = data['WCO0'] - data['WCO3']
data['Saving3'] = data['Saving3'].clip(lower=0)
data.loc[~data['Hour'].isin([6,7,8,16,17,18,19]),'Saving3'] = 0

In [8]:
df = data.copy()

# Calculate revenue for each offer
df['FXD0C'] = df.apply(lambda row: row['FXD0'] * (0.04449-row['P']), axis=1)
df['WCO0C'] = df.apply(lambda row: row['WCO0'] * (0.06319-row['P']), axis=1)
df['FXD1C'] = df.apply(lambda row: row['FXD1'] * (0.51967-row['P']) if row['Hour'] in [6, 7, 8] else row['FXD1'] * (0.04449-row['P']), axis=1)
df['WCO1C'] = df.apply(lambda row: row['WCO1'] * (0.06319-row['P']), axis=1)
df['FXD2C'] = df.apply(lambda row: row['FXD2'] * (0.51967-row['P']) if row['Hour'] in [16, 17, 18, 19] else row['FXD2'] * (0.04449-row['P']), axis=1)
df['WCO2C'] = df.apply(lambda row: row['WCO2'] * (0.06319-row['P']), axis=1)
df['FXD3C'] = df.apply(lambda row: row['FXD3'] * (0.51967-row['P']) if row['Hour'] in [6, 7, 8, 16, 17, 18, 19] else row['FXD3'] * (0.04449-row['P']), axis=1)
df['WCO3C'] = df.apply(lambda row: row['WCO3'] * (0.06319-row['P']), axis=1)
#calculate rebates HQ pays for WCO
df['S0'] = 0
df['S1'] = df.apply(lambda row: row['Saving1'] * 0.51967, axis=1)
df['S2'] = df.apply(lambda row: row['Saving2'] * 0.51967, axis=1)
df['S3'] = df.apply(lambda row: row['Saving3'] * 0.51967, axis=1)

# calculating hourly max loads under different rebates
sum_df = df.groupby(['DateTime'])[['FXD0C', 'WCO0C', 'FXD1C', 'WCO1C', 'FXD2C', 'WCO2C', 'FXD3C', 'WCO3C', 'S0', 'S1', 'S2', 'S3']].sum().resample('D').sum().reset_index()
# maximum daily records for aggregated loads
# this can only be used for not targeted options
max_df = df.copy()
max_df.set_index('DateTime', inplace=True)
max_df = max_df[['Load', 'WCO0', 'WCO1', 'WCO2', 'WCO3', 'FXD0', 'FXD1', 'FXD2', 'FXD3']].resample('H').sum().resample('D').max().reset_index()


merged_df = pd.merge(sum_df, max_df, on='DateTime')
merged_df['DateTime'] = pd.to_datetime(merged_df['DateTime'])


# Create nested dictionary from dataframe
nested_dict = {}

for index, row in merged_df.iterrows():
    dt = row['DateTime']
    year = dt.year
    month = dt.month
    day = dt.day
    
    if year not in nested_dict:
        nested_dict[year] = {}
    if month not in nested_dict[year]:
        nested_dict[year][month] = {}
    if day not in nested_dict[year][month]:
        nested_dict[year][month][day] = {}
    
    for col in merged_df.columns:
        if col != 'DateTime':
            nested_dict[year][month][day][col] = row[col]

In [6]:
# import pickle
# file = open('db.pkl', 'rb')
# db = pickle.load(file)

In [3]:
from home import Homes
ym = [[2013, 1], [2013, 2], [2013, 3], [2013, 12], 
    [2014, 1], [2014, 2], [2014, 3], [2014, 12], 
    [2015, 1], [2015, 2], [2015, 3], [2015, 12], 
    [2016, 1], [2016, 2], [2016, 3], [2016, 12], 
    [2017, 1], [2017, 2], [2017, 3], [2017, 12], 
    [2018, 1], [2018, 2], [2018, 3], [2018, 12],
    [2019, 1], [2019, 2], [2019, 3]]
scores = {}
for rate in [9999.00, 999.00, 499.00, 332.33, 199.00, 124.00, 110.11, 99.00, 65.67, 49.00, 32.33, 19.00, 9.00, 4.00, 1.00, 0.00]:
    print(f'rate: {rate}')
    env = Homes(tar= False, option= 'WCO', rate= rate)
    for year, month in ym:
        done = False
        _ = env.reset(dataset_mode= True, year= year, month= month)
        score = 0
        while not done:
            _, reward, done, _ = env.step(0)
            score += reward
        scores[rate, year, month] = score

rate: 9999.0
rate: 999.0
rate: 499.0
rate: 332.33
rate: 199.0
rate: 124.0
rate: 110.11
rate: 99.0
rate: 65.67
rate: 49.0
rate: 32.33
rate: 19.0
rate: 9.0
rate: 4.0
rate: 1.0
rate: 0.0


In [11]:
db = {}
db['d'] = d
db['data'] = data.copy()

db['nested_dict'] = nested_dict
db['scales'] = scores

with open('db.pkl', 'wb') as handle:
    pickle.dump(db, handle, protocol=pickle.HIGHEST_PROTOCOL)